In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import print_function, division
import os
import time
import copy
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, utils, models, datasets
import torch.nn as nn
import torch.optim as optim
import nibabel as nib
import scipy.ndimage as ndi
from pathlib import Path
from PIL import Image
import io

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
#imagesFolder = '/content/gdrive/MyDrive/Tesis/Imagenes/ADNI-MUESTRA-40'
#trainDatasetCSV = imagesFolder + '/ADNI-MUESTRA-40-UniformResolution_train.csv'
#valDatasetCSV = imagesFolder + '/ADNI-MUESTRA-40-UniformResolution_val.csv'

# muestra10 solo tiene train
imagesFolder = '/content/gdrive/MyDrive/Tesis/Imagenes/ADNI-MUESTRA-10'
trainDatasetCSV = imagesFolder + '/Muestra10UniformResolution.csv'
valDatasetCSV =   imagesFolder + '/Muestra10UniformResolution.csv'

crossEntrophyWeigths = torch.tensor([4.0,3.0,3.0])

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "inception"

# Number of classes in the dataset
num_classes = 3

# Batch size for training (change depending on how much memory you have)
batch_size = 2

# Number of epochs to train for
num_epochs = 50

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

usePretrained = True


In [ ]:
# https://stackoverflow.com/questions/8598673/how-to-save-a-pylab-figure-into-in-memory-file-which-can-be-read-into-pil-image
def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    buf = io.BytesIO()
    fig.savefig(buf, facecolor='black', dpi = 64, transparent=False) # dpi Requerido para que la imagen sea 512x512
    buf.seek(0)
    img = Image.open(buf)
    return img

In [ ]:
def transformGridImage(sample):
    brain_vol_data = sample.get_fdata()
    fig_rows = 4
    fig_cols = 4
    n_subplots = fig_rows * fig_cols
    n_slice = brain_vol_data.shape[2]
    step_size = n_slice // n_subplots
    plot_range = n_subplots * step_size
    start_stop = int((n_slice - plot_range) / 2)

    fig, axs = plt.subplots(fig_rows, fig_cols, figsize=[10, 10], facecolor='none', dpi=64)
    fig.set_size_inches(8, 8)
    fig.set_dpi(64)
    
    for idx, img in enumerate(range(start_stop, plot_range, step_size)):
        rotatedImg = ndi.rotate(brain_vol_data[:, :, img], 90)
        axs.flat[idx].imshow(np.squeeze(rotatedImg), cmap='gray') # np.squeeze es necesario para remover la dimension 1 al final
        axs.flat[idx].axis('off')
        
    plt.tight_layout()

    image = fig2img(fig)

    plt.close(fig) # Para que no muestre la imágen
    
    return image

In [ ]:
# CreateGrid transform
class CreateGrid(object):
    """Creates a grid from the image
    """
    def __init__(self):
        True

    def __call__(self, sample):
        return transformGridImage(sample)

In [ ]:
class RemoveTransparency(object):
    def __call__(self, sample):
      # La imagen se guarda con transparencia, removemos la dimension de indice 3
      return sample[0:3, : :]

In [ ]:
class ToLabelOutput(object):
    def __call__(self, label):
        if label == "CN":
            return 0
        elif label == "AD":
            return 1
        else:
            return 2 # MCI, LMCI, EMCI

In [ ]:
class ADNIDataset(Dataset):
    """ADNI dataset."""

    def __init__(self, csv_file, root_dir, transform=None, target_transform = None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.csv = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
      return int(len(self.csv))

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        studyID = self.csv.iloc[idx, 0]
        subjectID = self.csv.iloc[idx, 1]
        processFormat = self.csv.iloc[idx, 7]
        date = self.csv.iloc[idx, 9]
        diagnosis = self.csv.iloc[idx, 2]
        
        filename = None
        
        rglob = str(studyID)+'/'+'*.nii'
        samples = 0
        # for path in Path('ADNI-Full-PostProc').rglob(rglob):
        for path in Path(self.root_dir).rglob(rglob):
            filename = str(path)
            samples =+ 1
            
        if samples > 1:
            raise "Mas de un sample. Error"

        if not filename:
            raise Exception("Not found filename for index " + str(idx) + " y studyID " + studyID)

        # print("Leyendo: " + filename)
            
        brain_vol = nib.load(filename)

        image = brain_vol
        label = diagnosis
        
        if self.transform:
            image = self.transform(image)
            
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [ ]:
adniDataset = ADNIDataset('/content/gdrive/MyDrive/Tesis/Imagenes/ADNI-MUESTRA-40/ADNI-MUESTRA-40-UniformResolution.csv', '/content/gdrive/MyDrive/Tesis/Imagenes/ADNI-MUESTRA-40', transform = CreateGrid())
len(adniDataset)

In [ ]:
image, label = adniDataset[38]
label

# Con transformacion a tensor

In [ ]:
adniDataset = ADNIDataset('/content/gdrive/MyDrive/Tesis/Imagenes/ADNI-MUESTRA-40/ADNI-MUESTRA-40-UniformResolution.csv', '/content/gdrive/MyDrive/Tesis/Imagenes/ADNI-MUESTRA-40', transform = transforms.Compose([CreateGrid(), transforms.ToTensor()]), target_transform =ToLabelOutput() )
len(adniDataset)

In [ ]:
image, label = adniDataset[2]
label

# Modelo

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=True):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        # No usamos el aux
                        #outputs, aux_outputs = model(inputs)
                        #loss1 = criterion(outputs, labels)
                        #loss2 = criterion(aux_outputs, labels)
                        #loss = loss1 + 0.4*loss2
                        outputs, aux_outputs = model(inputs)
                        loss = criterion(outputs, labels)
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# Initialize and reshape inception

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained, 
                                       aux_logits = True)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        # num_ftrs = model_ft.AuxLogits.fc.in_features
        # model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        print("num featurs" + str(num_ftrs))
        # Fuente: https://github.com/bdrad/petdementiapub/blob/master/petdementia_source.py
        model_ft.fc = nn.Sequential(
          nn.Linear(num_ftrs,1024),
          nn.ReLU(),
          nn.Linear(1024,num_classes),
          # nn.Softmax() # innecesario, cross entrophy acepta logits
        )
          
        input_size = 512 

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=usePretrained)

# Print the model we just instantiated
# print(model_ft)

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(input_size),
        #transforms.RandomHorizontalFlip(),
        CreateGrid(),
        transforms.ToTensor(),
        RemoveTransparency(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(input_size),
        #transforms.CenterCrop(input_size),
        CreateGrid(),
        transforms.ToTensor(),
        RemoveTransparency(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets

image_datasets = {
    'train': ADNIDataset(trainDatasetCSV, imagesFolder, transform = data_transforms['train'], target_transform =ToLabelOutput() ),
    'val': ADNIDataset(valDatasetCSV, imagesFolder, transform = data_transforms['val'], target_transform =ToLabelOutput() )
}

# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=0.0001)


In [ ]:
# Setup the loss fxn
crossEntrophyWeigths = crossEntrophyWeigths.to(device)
criterion = nn.CrossEntropyLoss(crossEntrophyWeigths)

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

# Prueba manual para ver qué devuelve

In [ ]:
dataset = ADNIDataset(valDatasetCSV, imagesFolder, transform = transforms.Compose([CreateGrid(), transforms.ToTensor(), RemoveTransparency()]), target_transform =ToLabelOutput() )

# Create training and validation dataloaders
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

i = 0
for inputs, labels in dataloader:
  inputs = inputs.to(device)
  labels = labels.to(device)
  print("Index: " + str(i))
  print(labels)
  print("Result: ")
  print(model_ft(inputs))
  i += 1